In [1]:
import pandas as pd
from pathlib import Path
import IMP
import IMP.atom
import IMP.rmf
import RMF
import random
import numpy as np
import matplotlib.pyplot as plt


In [6]:
gsms_dir = Path(Path.home(), "Documents/mtorc2/tmp/10000R")
gsms_file = Path(gsms_dir, "A.rmf3")
fh_clust = RMF.open_rmf_file_read_only(str(gsms_file))
m_clust = IMP.Model()
h_clust = IMP.rmf.create_hierarchies(fh_clust, m_clust)[0]

df_file = Path(Path.home(), "Documents/mtorc2/dev/11_ph_analysis/data/A_ph.csv")


In [7]:
def find_normal_vector(a, b, c):
    # Convert points to numpy arrays
    A = np.array(a)
    B = np.array(b)
    C = np.array(c)

    # Calculate vectors AB and AC
    AB = B - A
    AC = C - A

    # Calculate the cross product of AB and AC
    normal = np.cross(AB, AC)

    return normal

def find_unit_normal_vector(a, b, c):
    normal = find_normal_vector(a, b, c)
    magnitude = np.linalg.norm(normal)
    unit_normal = normal / magnitude
    return unit_normal

def euclidean_distance(a, b):
    return np.linalg.norm(np.array(a) - np.array(b))

In [8]:
df_data = pd.DataFrame(index=range(10000),columns=["MSIN1_l2", "AKT1_l2", "d_MSIN1", "d_AKT1"])

res_dict = {"MSIN1": [394, 463, 429], "AKT1": [53, 18, 38]}
for i in range(10000):
    print(i)
    IMP.rmf.load_frame(fh_clust, RMF.FrameID(i))

    for mol in ["MSIN1", "AKT1"]:
        sel = IMP.atom.Selection(h_clust, resolution=1, molecule=mol, residue_indexes=res_dict[mol], copy_index=0)
        pids = sel.get_selected_particles()

        norm = find_normal_vector(IMP.core.XYZ(m_clust, pids[0]).get_coordinates(), IMP.core.XYZ(m_clust, pids[1]).get_coordinates(), IMP.core.XYZ(m_clust, pids[2]).get_coordinates())

        unit_norm = find_unit_normal_vector(IMP.core.XYZ(m_clust, pids[0]).get_coordinates(), IMP.core.XYZ(m_clust, pids[1]).get_coordinates(), IMP.core.XYZ(m_clust, pids[2]).get_coordinates())

        l2 = euclidean_distance(unit_norm, np.array([0,0,-1]))
        df_data.loc[i, "{}_l2".format(mol)] = l2

    print(unit_norm, l2)

df_data.to_csv(Path(gsms_dir, "A_ph.csv"))

0
[ 0.6559835  -0.04523095  0.75341875] 1.8726551989502103
1
[ 0.68514084 -0.51661383  0.51350967] 1.7398331348511509
2
[ 0.04271663 -0.91442468  0.40249571] 1.6748108635755476
3
[ 0.88064066 -0.47273028 -0.03159283] 1.3916947695066613
4
[-0.53302929 -0.79551565 -0.28815729] 1.1931828931734452
5
[ 0.88064066 -0.47273028 -0.03159283] 1.3916947695066613
6
[-0.07588992 -0.87411205 -0.47975916] 1.0200400387400415
7
[ 0.35332693 -0.56128908  0.74840808] 1.8699775836023336
8
[ 0.17707364 -0.75088067  0.63625713] 1.8090091952014906
9
[-0.20180828  0.08894093  0.97537835] 1.9876510521617377
10
[-0.34064849 -0.80349565 -0.48821445] 1.011716903886074
11
[ 0.01944767  0.17614474 -0.98417215] 0.17792049808765031
12
[-0.10497961  0.70352117 -0.70287783] 0.7708724518833662
13
[-0.2362436  -0.95934243 -0.15443787] 1.3004323344762216
14
[ 0.08293947 -0.94391791 -0.31959354] 1.166538863313888
15
[ 0.60441009 -0.79666558  0.00352215] 1.416701908353582
16
[ 0.61498772 -0.47772267  0.6273525 ] 1.804080098

In [9]:
for i in range(10000):
    IMP.rmf.load_frame(fh_clust, RMF.FrameID(i))

    for mol in ["MSIN1", "AKT1"]:
        if mol == "MSIN1":
            lower_bound, upper_bound = 383,522
        elif mol == "AKT1":
            lower_bound, upper_bound = 1,143

        sel = IMP.atom.Selection(h_clust, resolution=1, molecule=mol, residue_indexes=list(range(lower_bound,upper_bound)), copy_index=0)

        pids = [pid for pid in sel.get_selected_particle_indexes()]

        xyzs = [IMP.core.XYZ(m_clust, pid) for pid in pids]

        cum_dist = 0
        for xyz in xyzs:
            z_coord = xyz.get_coordinates()[2]
            cum_dist = cum_dist + (244.481 - z_coord)

        avg_dist = cum_dist / len(xyzs)
        df_data.iloc[i]["d_{}".format(mol)] = avg_dist

# del fh_clust


In [14]:
# df_data.to_csv(df_file)

df_data = pd.read_csv(df_file, index_col=0)
df_data.head()

,MSIN1_l2,AKT1_l2,d_MSIN1,d_AKT1
0,1.786761,1.872655,24.651164,67.801792
1,1.918555,1.739833,43.454890,59.187009
2,0.942075,1.674811,39.550495,47.062708
3,1.759234,1.391695,75.082306,70.173938
4,1.048700,1.193183,80.888812,53.644783


In [18]:
df_data["MSIN1_l2"] = df_data["MSIN1_l2"].astype(float)
df_data["AKT1_l2"] = df_data["AKT1_l2"].astype(float)
df_data["l2_total"] = df_data["MSIN1_l2"] + df_data["AKT1_l2"]

In [19]:
df_data["d_AKT1"] = pd.to_numeric(df_data["d_AKT1"])
df_data["d_MSIN1"] = pd.to_numeric(df_data["d_MSIN1"])

df_data["d_total"] = df_data["d_AKT1"] + df_data["d_MSIN1"]


In [20]:
df_data.head()

,MSIN1_l2,AKT1_l2,d_MSIN1,d_AKT1,d_total,l2_total
0,1.786761,1.872655,24.651164,67.801792,92.452955,3.659416
1,1.918555,1.739833,43.454890,59.187009,102.641898,3.658388
2,0.942075,1.674811,39.550495,47.062708,86.613203,2.616886
3,1.759234,1.391695,75.082306,70.173938,145.256244,3.150929
4,1.048700,1.193183,80.888812,53.644783,134.533595,2.241883


In [40]:
sorted_df = df_data.sort_values(by='MSIN1_l2', ascending=True)

# Get the 50th to 100th largest values
result_df = sorted_df.iloc[0:50]
result_df.head(50)

# df_data.nlargest(100, "d_MSIN1").nsmallest(50, "d_MSIN1")

,MSIN1_l2,AKT1_l2,d_MSIN1,d_AKT1,d_total,l2_total
3402,0.009137,1.064241,17.146422,68.706384,85.852806,1.073379
6305,0.027952,1.547560,57.808801,45.849677,103.658478,1.575512
5468,0.040567,1.167142,57.072850,73.237571,130.310421,1.207710
752,0.045005,1.300229,53.352721,58.195568,111.548289,1.345234
4605,0.054605,1.044918,13.378159,79.810533,93.188692,1.099522
7291,0.058401,1.113378,54.662401,72.947407,127.609808,1.171780
8424,0.059449,0.760735,2.986266,92.454321,95.440587,0.820184
4229,0.060386,1.493015,4.486215,71.146704,75.632919,1.553401
6008,0.060627,1.781738,77.875175,56.843531,134.718706,1.842365
9031,0.060770,0.685940,38.338690,70.926890,109.265580,0.746710


In [28]:
# df_data.nsmallest(50, "MSIN1_l2")

In [41]:
frame_id = 1280
IMP.rmf.load_frame(fh_clust, RMF.FrameID(frame_id))
sel = IMP.atom.Selection(h_clust, resolution=1, molecules=["MSIN1", "AKT1"])
IMP.atom.write_pdb(sel, str(Path(Path.home(), "Documents/mtorc2/tmp/{}".format(str(frame_id)+".pdb"))))

In [17]:
pdb_dir = Path(Path.home(), "Documents/mtorc2/tmp/membrane")
for frame_id in list(pd.DataFrame(df_data).nsmallest(25, "d_AKT1").index):
    print(frame_id)
    IMP.rmf.load_frame(fh_clust, RMF.FrameID(frame_id))

    sel = IMP.atom.Selection(h_clust, resolution=1, molecules=["MSIN1", "AKT1"])

    IMP.atom.write_pdb(sel, str(Path(pdb_dir, str(frame_id)+".pdb")))



775


IOException: Unable to open file /Users/matthew/Documents/mtorc2/tmp/membrane/775.pdb
